In [1]:
from __future__ import annotations

import os 

from torch.utils.data import random_split

from mmpfn.datasets.pad_ufes_20 import PADUFES20Dataset

import os 
import torch 
import numpy as np 
import pandas as pd

from sklearn.metrics import accuracy_score
from mmpfn.models.mmpfn_v2 import MMPFNClassifier
from mmpfn.models.mmpfn_v2.constants import ModelInterfaceConfig
from mmpfn.models.mmpfn_v2.preprocessing import PreprocessorConfig
from mmpfn.scripts_finetune_mm.finetune_tabpfn_main import fine_tune_mmpfn

/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/wall/works/research/MultiModalPFN/mmpfn/models/dino_v2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
image_type='cls' # patch cls
# data_path = os.path.join(os.getenv('HOME'), "workspace/works/tabular_image/MultiModalPFN/mmpfn/data/pad_ufes_20")
data_path = os.path.join(os.getenv('HOME'), "works/research/MultiModalPFN/mmpfn/data/pad_ufes_20")
dataset = PADUFES20Dataset(data_path)
_ = dataset.get_images()
_ = dataset.get_embeddings()

In [ ]:
accuracy_scores = []
for seed in range(10):
    torch.manual_seed(seed)
    train_len = int(len(dataset) * 0.8)
    test_len = len(dataset) - train_len

    train_dataset, test_dataset = random_split(dataset, [train_len, test_len])

    X_train = train_dataset.dataset.x[train_dataset.indices]
    y_train = train_dataset.dataset.y[train_dataset.indices]
    X_test = test_dataset.dataset.x[test_dataset.indices]
    y_test = test_dataset.dataset.y[test_dataset.indices]
    image_train = train_dataset.dataset.embeddings[train_dataset.indices]
    image_test = test_dataset.dataset.embeddings[test_dataset.indices]

    # if image_type == 'cls':
    #     image_train = image_train.unsqueeze(1)
    #     image_test = image_test.unsqueeze(1)
        
    for i in range(X_train.shape[1]):
        col = X_train[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1
    for i in range(X_test.shape[1]):
        col = X_test[:, i]
        col[np.isnan(col)] = np.nanmin(col) - 1

    torch.cuda.empty_cache()

    save_path_to_fine_tuned_model = "./finetuned_mmpfn_pad_ufes_20.ckpt"
    
    fine_tune_mmpfn(
        # path_to_base_model="auto",
        save_path_to_fine_tuned_model=save_path_to_fine_tuned_model,
        # Finetuning HPs
        time_limit=60,
        finetuning_config={"learning_rate": 0.00001, "batch_size": 1, "max_steps": 100},
        validation_metric="log_loss",
        # Input Data
        X_train=pd.DataFrame(X_train),
        image_train=image_train,
        y_train=pd.Series(y_train),
        categorical_features_index=None,
        device="cuda",  # use "cpu" if you don't have a GPU
        task_type="multiclass",
        # Optional
        show_training_curve=False,  # Shows a final report after finetuning.
        logger_level=0,  # Shows all logs, higher values shows less
        freeze_input=True,  # Freeze the input layers (encoder and y_encoder) during finetuning
        mixer_type='MGM+CQAM', # MGM MGM+CQAM
        mgm_heads=32,
        cqam_heads=8
    )

    # disables preprocessing at inference time to match fine-tuning
    no_preprocessing_inference_config = ModelInterfaceConfig(
        FINGERPRINT_FEATURE=False,
        PREPROCESS_TRANSFORMS=[PreprocessorConfig(name='none')]
    )

    # Evaluate on Test Data
    model_finetuned = MMPFNClassifier(
        model_path=save_path_to_fine_tuned_model,
        inference_config=no_preprocessing_inference_config,
        ignore_pretraining_limits=True,
        mixer_type='MGM+CQAM', # MGM MGM+CQAM
        mgm_heads=32,
        cqam_heads=8
    )

    clf_finetuned = model_finetuned.fit(X_train, image_train, y_train)
    acc_score = accuracy_score(y_test, clf_finetuned.predict(X_test, image_test))
    print("accuracy_score (Finetuned):", acc_score)
    accuracy_scores.append(acc_score)

Fine-tuning Steps:   8%|▊         | 8/100 [00:05<01:09,  1.33it/s, Best Val. Loss=0.564, Best Val. Score=-0.564, Training Loss=0.654, Val. Loss=0.564, Patience=43, Utilization=0, Grad Norm=8.15][2025-09-12 23:21:00,597] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  20%|██        | 20/100 [00:14<00:56,  1.41it/s, Best Val. Loss=0.537, Best Val. Score=-0.537, Training Loss=0.543, Val. Loss=0.539, Patience=32, Utilization=0, Grad Norm=11.5][2025-09-12 23:21:09,469] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps:  44%|████▍     | 44/100 [00:30<00:35,  1.56it/s, Best Val. Loss=0.512, Best Val. Score=-0.512, Training Loss=0.454, Val. Loss=0.533, Patience=9, Utilization=0, Grad Norm=31.6] [2025-09-12 23:21:25,616] INFO - 
Optimizer step skipped due to NaNs/infs in grad scaling.
Fine-tuning Steps: 101it [01:11,  1.40it/s, Best Val. Loss=0.493, Best Val. Score=-0.493, Training Loss=0.459, Val. Loss=0.495, Patience=-47, U

accuracy_score (Finetuned): 0.8608695652173913


Fine-tuning Steps:   5%|▌         | 5/100 [00:02<01:01,  1.54it/s, Best Val. Loss=0.629, Best Val. Score=-0.629, Training Loss=0.588, Val. Loss=0.644, Patience=46, Utilization=0, Grad Norm=5.42]


KeyboardInterrupt: 

In [ ]:
# get mean and std of accuracy scores
mean_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)
print("Mean Accuracy:", mean_accuracy)
print("Std Accuracy:", std_accuracy)

Mean Accuracy: 0.8380434782608696
Std Accuracy: 0.01706902477604662
